**Imports**

Data

In [3]:
#enter the path to get to your CDs_and_Vinyl folder below
# for me (max) The data is in the repo's parent folder
PATH_TO_CDS = '../devided_dataset_v2/CDs_and_Vinyl/'

import pandas as pd

# Open and load json training files
x = pd.read_json(PATH_TO_CDS + 'train/review_training.json')
y = pd.read_json(PATH_TO_CDS + 'train/product_training.json')

ML

In [21]:
import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split

from sklearn.naive_bayes import GaussianNB
gnb = GaussianNB()

from sklearn.feature_extraction.text import TfidfVectorizer
tfidf = TfidfVectorizer()
from nltk import sentiment

**Pre Processing**

In [ ]:
#z = pd.merge(x,y,'inner','asin')
#zgrouped = z.groupby("asin")

In [ ]:
x #770786 entries
y #073082 entries

,asin,awesomeness
0,0000B049F5B33CD310EB1AB236E20191,1
1,00018184A9EC4D270219A296B2580303,0
2,000281A9CAC43FF1F335726A390636DA,0
3,00030884DF109F325638A6BFD5B13CFF,1
4,000325BA25966B5FC701D5D2B5DBA4E0,0
...,...,...
73077,FFFDD3C72D23AF858D6E0ED92612370D,1
73078,FFFDDE284A73B29B320381487EC7DE9E,1
73079,FFFEB3EE2372807964F024707D50FB21,0
73080,FFFF4545AB232D81D0F9B208388BB7AA,1


Feature creation testing

Making a single feature ex

In [5]:
reviewCount = x.groupby('asin')["reviewerID"].count()
reviewCount

asin
0000B049F5B33CD310EB1AB236E20191     4
00018184A9EC4D270219A296B2580303    18
000281A9CAC43FF1F335726A390636DA     4
00030884DF109F325638A6BFD5B13CFF    29
000325BA25966B5FC701D5D2B5DBA4E0     4
                                    ..
FFFDD3C72D23AF858D6E0ED92612370D    64
FFFDDE284A73B29B320381487EC7DE9E     4
FFFEB3EE2372807964F024707D50FB21     2
FFFF4545AB232D81D0F9B208388BB7AA     7
FFFF5A3D9CB0B40FF0FE6B95F05D26FE    27
Name: reviewerID, Length: 73082, dtype: int64

In [6]:
test2 = x.groupby('asin')['unixReviewTime'].mean()


Dropping ex

In [ ]:
dropped = x[x.verified != True]
dropped

Merging tests

In [8]:
z = pd.merge(reviewCount,test2,'inner','asin')

z

,reviewerID,unixReviewTime
asin,,
0000B049F5B33CD310EB1AB236E20191,4,1.296583e+09
00018184A9EC4D270219A296B2580303,18,1.111282e+09
000281A9CAC43FF1F335726A390636DA,4,1.254960e+09
00030884DF109F325638A6BFD5B13CFF,29,1.315392e+09
000325BA25966B5FC701D5D2B5DBA4E0,4,1.363802e+09
...,...,...
FFFDD3C72D23AF858D6E0ED92612370D,64,1.258704e+09
FFFDDE284A73B29B320381487EC7DE9E,4,1.353758e+09
FFFEB3EE2372807964F024707D50FB21,2,1.349050e+09


groupby() testing

In [49]:
z = pd.merge(x,y,on="asin")
zgrouped = z.groupby("asin")
print(len(z))
z = z[z.verified == True]
print(len(z))

770786
401407


In [38]:
## Helper function for sentiment
def review_sentiment(text):
    sia = sentiment.SentimentIntensityAnalyzer()
    return sia.polarity_scores(text)["compound"]

## just some examples to gut check
print(review_sentiment("Ugh, what a terrible, horrible, no good, very bad day"))
print(review_sentiment("I went to the store today"))
print(review_sentiment("I absolutely love my favorite best friend"))

-0.9099
0.0
0.9447


In [47]:

tests = range(1)
unique_items = z["asin"].nunique()
print("ui: ", unique_items)

asin = z["asin"].unique()
# print("asin: ",asin[tests])
avg_votes = np.zeros(len(asin))

for i in tests: # eventually all
    item = zgrouped.get_group(asin[i])
    avg_votes[i] = item["vote"].fillna(0).astype(int).mean()
    sentiments = list(map(review_sentiment,item["reviewText"].values))
    print(sentiments)

print("avg_votes: ", avg_votes[0:5])
    


ui:  73082
14577


KeyboardInterrupt: 

**Testing**

Test example: 

*Gaussian Naive Bayes with tfidf*

In [ ]:
#split data
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.5, random_state=0)

#train and apply tfidf
    # fit = make vocab 
    # transform = turn words -> numbers
x_train = tfidf.fit_transform(x_train).toarray()
x_test = tfidf.transform(x_test).toarray()

#make prediction
y_pred = gnb.fit(x_train, y_train).predict(x_test)

#print accuracy
print(f"Number of mislabeled points out of a total {x_test.shape[0]} points : {(y_test != y_pred).sum()}")